# クイックスタート

このガイドでは minto を活用するための初期手順を案内します。このセクションでは、minto のセットアップから実験データの取り扱いまでの 4 つの主要なステップを説明します。

- minto のインストール方法
- 実験データの記録
- 記録された実験データのテーブル表示
- 実験データの保存と読み込み

## インストール

```bash
pip install minto
```

## 実験データの記録

minto を使うと、実験中に生成されるさまざまなデータを簡単に記録できます。
次の例では、OMMX でサポートされている PySCIPOpt を用いて、MIP ソルバーのタイムリミット依存性について数値実験を行います。
minto は OMMX メッセージをネイティブにサポートしているため、OMMX を通じてスムーズに数値実験を実施できます。

さっそく試してみましょう。

In [ ]:
import minto
import ommx_pyscipopt_adapter as scip_ad
from ommx.dataset import miplib2017

このチュートリアルでは、ベンチマークとして miplib2017 のインスタンスを取り上げます。ommx.dataset を使用して、miplib2017 のインスタンスを簡単に取得できます。

In [ ]:
instance_name = "reblock115"
instance = miplib2017(instance_name)

`ommx_pyscipopt_adapter` を使用して、ommx.v1.Instance を PySCIPOpt の Model に変換し、limits/time パラメータを変化させながら実験を行います。

ommx インスタンスとソリューションは、MINTO の `.log_*` メソッドで保存できます。この数値実験では単一のインスタンスを使用しており、実験全体で変化がないため、with ブロックの外側（`experiment` 空間）でインスタンスを保存します。
タイムリミットごとにソリューションが異なるため、ソリューションはwith ブロック内（`run` 空間）で保存します。

In [ ]:
timelimit_list = [0.1, 0.5, 1, 2]

# auto_saving が True の場合、Experiment.log_* メソッドが呼び出されるたびにデータが自動的に保存されます。
# データは段階的に保存されるため、途中でエラーが発生しても、それまでのデータは失われません。
experiment = minto.Experiment(
    name='scip_exp',
    auto_saving=True
)

experiment.log_instance(instance_name, instance)
adapter = scip_ad.OMMXPySCIPOptAdapter(instance)
scip_model = adapter.solver_input

for timelimit in timelimit_list:
    with experiment.run():
        experiment.log_parameter("timelimit", timelimit)

        # SCIP による求解を実行
        scip_model.setParam("limits/time", timelimit)
        scip_model.optimize()
        solution = adapter.decode(scip_model)

        experiment.log_solution("scip", solution)

ommx.Solution を pandas.DataFrame に変換する際、`.get_run_table` メソッドを利用すると、ソリューションの主要情報のみが表示されます。実際のソリューションオブジェクトにアクセスしたい場合は、`experiment.dataspaces.run_datastores[run_id].solutions` から参照してください。

In [ ]:
runs_table = experiment.get_run_table()
runs_table

In [ ]:
import matplotlib.pyplot as plt

x = runs_table["parameter", "timelimit"]
y = runs_table["solution_scip", "objective"]
plt.plot(x, y, "o-")
plt.xlabel("Time limit")
plt.ylabel("Objective")
plt.show()

## 実験データの保存と読み込み

`Experiment` コンストラクタで `auto_saving` を `False` に設定すると、自動保存を無効にできます。この場合、`Experiment.save` メソッドを使って任意のタイミングでデータを保存できます。
また、`Experiment.load_from_dir` メソッドを使って、保存したデータを読み込むことも可能です。

In [ ]:
# デフォルトでは、データは .minto_experiments/ ディレクトリに保存されます。
experiment.save()

In [ ]:
exp2 = minto.Experiment.load_from_dir(
    ".minto_experiments/" + experiment.experiment_name
)

In [ ]:
exp2.get_run_table()

## OMMX アーカイブへの保存と読み込み

`Experiment.save_to_ommx_archive` および `Experiment.load_from_ommx_archive` メソッドを使用することで、OMMX アーカイブへのデータの保存と、OMMX アーカイブからのデータの読み込みが可能です。

In [ ]:
artifact = experiment.save_as_ommx_archive("scip_experiment.ommx")

In [ ]:
exp3 = minto.Experiment.load_from_ommx_archive("scip_experiment.ommx")
exp3.get_run_table()

# まとめ

このチュートリアルでは、数値実験を管理するための `minto` の使い方を学びました:

1. **インストール** はpip を使って簡単に実行できます:
    ```bash
    pip install minto
    ```

2. **実験データの記録**
    - OMMX インターフェースを介した SCIP ソルバーの使用
    - 0.1 秒から 2.0 秒まで異なるタイムリミットで実験を実施
    - インスタンス情報とソリューションの結果を体系的に記録

3. **データ管理**
    - `auto_saving=True` による自動保存
    - `experiment.save()` による手動保存
    - pandas DataFrame と matplotlib を用いたデータの可視化
    - 結果では目的関数値が -2.92e+07 に収束

4. **データの永続化**
    - `.minto_experiments/` ディレクトリへのローカル保存
    - OMMX アーカイブ形式サポート
    - `load_from_dir` や `load_from_ommx_archive` で簡単にデータを復元

これらの手順によって、最適化タスクにおける `minto` の構造的な実験管理機能が示されました。